# Fisher Matrix for Weak Lensing Observable

The next code is the **draft final code** and must be optimized (specifically to change the functions for each type of cosmology). 

In each cosmology I print the Fisher Matrix and the errors for each parameter considered.

In [49]:
import numpy as np
import sympy as smp
from scipy import integrate
import matplotlib.pyplot as plt
from torchquad import Simpson, set_up_backend
from scipy.integrate import quad, nquad
import inspect
import pandas as pd

# For interpolation
from scipy.interpolate import RectBivariateSpline, interp2d

import warnings

# Ignorar solo DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message=".*divmax.*")

import logging
# Configuración básica del registro
logging.basicConfig(level=logging.INFO)

## Cosmological constants


In [50]:
# First model
Omega_b0= 0.05
omega_b0= 0.022445
Omega_m0 = 0.32
omega_m0 = 0.143648
h = 0.67
n_s = 0.96
sigma_8 = 0.816
Omega_DE0 = 0.68
w_0 = -1
w_a = 0
gamma = 0.55
Omega_K0 = 1 - (Omega_DE0 + Omega_m0) # FLAT universe!

#[params_cosmo]
#sigma8 = 0.815584
#h = 0.670
#omega_cdm      = 0.2685628338348412
#omega_lambda   = 0.68
#pivot_scalar              = 0.05
#pivot_tensor              = 0.05
#scalar_amp(1)             = 2.12605e-09
#scalar_spectral_index(1)  = 0.96
#Omega_b = 0.05
#Omega_m = 0.32
#Omega_nu = 0.00143717
#n_s = 0.96
#A_s = 2.12605e-09

# Euclid specifications
sigma_epsilon  = 0.3
n_g = 30 #arcmin^-2. Surface density of galaxies
A_survey = 15000 #deg^2
N_z= 10 #number of redshift bins
ng_i = n_g/N_z # Surface density of galaxies per bin

## Import the data set

In [51]:
# Matter power spectrum
P = 'c:/Users/MICROSOSFT/Documents/Investigacion 1/fiducial_eps_0/Pnonlin-zk.txt'
P_deltas = pd.read_csv(P, header=None, delimiter=' ')  # Cambia ' ' por el delimitador correcto

P_array_2d = P_deltas.values
P_list = P_deltas.values.flatten().tolist()
P_array_1d = np.array(P_list) # DONT USE

# Redshift
red = 'c:/Users/MICROSOSFT/Documents/Investigacion 1/fiducial_eps_0/z_values_list.txt'
z = pd.read_csv(red, header=None, delimiter=' ')  # Cambia ' ' por el delimitador correcto

z_list = z.values.flatten().tolist()
z_array = np.array(z_list)

# k values
kas = 'c:/Users/MICROSOSFT/Documents/Investigacion 1/fiducial_eps_0/k_values_list.txt'
k = pd.read_csv(kas, header=None, delimiter=' ')  # Cambia ' ' por el delimitador correcto

k_list = k.values.flatten().tolist()
k_array = np.array(k_list)

## Fiducial values

In [52]:
Omega_c0 = 0.2685628338348412

Omega_nu0 = 0.00143717

Omega_b0 = 0.05

Omega_m0 = Omega_c0 + Omega_nu0 + Omega_b0

Omega_DE0 = Omega_lambda = 0.68

w0= -1

wa = 0

Omega_K0 = 1 - (Omega_m0 + Omega_DE0)

h = 0.67

c = 299792

sigma8 = 0.816

ns = 0.96

gamma = 6/11

### Numerical Derivatives

In [53]:
## Derivada con respecto a ns y sigma8

# Matter power spectrum_plus_ns
P_pl_ns_data = 'c:/Users/MICROSOSFT/Documents/Investigacion 1/ns_pl_eps_1p0E-2/Pnonlin-zk.txt'
P_deltas_pl_ns = pd.read_csv(P_pl_ns_data, header=None, delimiter=' ')  

P_pl_ns = P_deltas_pl_ns.values

# Matter power spectrum_minor_ns
P_mn_ns_data = 'c:/Users/MICROSOSFT/Documents/Investigacion 1/ns_mn_eps_1p0E-2/Pnonlin-zk.txt'
P_deltas_mn_ns = pd.read_csv(P_mn_ns_data, header=None, delimiter=' ')  

P_mn_ns = P_deltas_mn_ns.values

# Matter power spectrum_plus_sigma8
P_pl_sig_data = 'c:/Users/MICROSOSFT/Documents/Investigacion 1/s8_pl_eps_1p0E-2/Pnonlin-zk.txt'
P_deltas_pl_sig = pd.read_csv(P_pl_sig_data, header=None, delimiter=' ')  

P_pl_sig = P_deltas_pl_sig.values

# Matter power spectrum_minor_sigma8
P_mn_sig_data = 'c:/Users/MICROSOSFT/Documents/Investigacion 1/s8_mn_eps_1p0E-2/Pnonlin-zk.txt'
P_deltas_mn_sig = pd.read_csv(P_mn_sig_data, header=None, delimiter=' ')  

P_mn_sig = P_deltas_mn_sig.values

In [54]:
# Derivadas numericas

der_P_ns = (P_pl_ns - P_mn_ns)/(2*0.01*ns)

der_P_sigma8 = (P_pl_sig - P_mn_sig)/(2*0.01*sigma8)

# $\Lambda CDM$ flat

Parameters:

1. $\Omega_{m,0}$

2. $h$

3. $\Omega_{b,0}$

4. $\Omega{DE,0}$

5. $n_s$

6. $\sigma_9$

**Considerations**

1. Adding intrinsic aligment (for now F =1)

        a. L?

In [55]:
class Class_integration:
    '''
    Class to calculate the Windows Function
    '''
    def __init__(self, params):
        self.len = params['len']
        self.z = params['z']

    def E2(self, z, w0, wa, Omega_m0, Omega_b0, Omega_DE0):
        Oomega_m0 = 2*(Omega_c0 + Omega_nu0 + Omega_b0) - (Omega_m0)
        Omega_k0 = 1 - (Oomega_m0+Omega_DE0)
        radicando = (Oomega_m0)*(1+z)**3 + Omega_DE0*(1+z)**(3*(1+wa+w0))*np.exp(-3*wa*(z / (1+z))) + (Omega_k0)*(1+z)**2
        return np.sqrt(radicando)
    
    def inverse_E2(self, z, w0, wa, Omega_m0, Omega_b0, Omega_DE0):
        return 1 / self.E2(z, w0, wa, Omega_m0, Omega_b0, Omega_DE0)
    
    def n_t(self, z):
        z_m = 0.9
        z_0 = z_m/np.sqrt(2)
        
        return ((z/z_0)**2)*np.exp(-(z/z_0)**(3/2))

    # Probability function

    def p_ph(self, z_p, z):
        c_b = 1
        z_b = 0
        sigma_b = 0.05
        c_o = 1
        z_o = 0.1
        sigma_o = 0.05
        f_out = 0.1

        first = ((1-f_out)/(np.sqrt(2*np.pi)*sigma_b*(1+z)))*np.exp(-0.5*((z - (c_b*z_p) - z_b)/(sigma_b*(1+z)))**2)
        second = (f_out/(np.sqrt(2*np.pi)*sigma_o*(1+z)))*np.exp(-0.5*((z - (c_o*z_p) - z_o)/(sigma_o*(1+z)))**2)
        return first + second
        
    def r(self, z, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h):
        H_0 = 100*h
        z_prime = np.linspace(0, z, self.len)
        delta = (z) / len(z_prime)
        integrand = self.inverse_E2(z_prime, w0, wa, Omega_m0, Omega_b0, Omega_DE0)*delta
        return np.sum(integrand)*(c/H_0)
    
    def n_i_try(self, i, z):
        z_bins = [0.001, 0.42, 0.56, 0.68, 0.79, 0.9, 1.02, 1.15, 1.32, 1.58, 2.5]
        
        denominators = np.array([0.04599087, 0.04048852, 0.04096115, 0.03951212, 0.03886145,
        0.03944441, 0.03751183, 0.03950185, 0.04042198, 0.03827518])
        
        def numerator_n_i(i, z):
            z_minor_i = z_bins[i]
            z_major_i = z_bins[i + 1]
            z_prime = np.linspace(z_minor_i, z_major_i, self.len)
            delta = (z_major_i - z_minor_i)/len(z_prime)

            multiplication_array = self.n_t(z)*self.p_ph(z_prime, z)
            
            return np.sum(multiplication_array*delta)
        
        numerator_values = numerator_n_i(i,z) 
        denominator_value = denominators[i]
        return numerator_values / denominator_value
    
    def Window2(self, i, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h):
        result = []
        for z in self.z:
            z_max = 2.5
            z_prime = np.linspace(z, z_max, self.len)
            delta = (z_max - z) / len(z_prime)
            r_true = self.r(z, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h)
            n_array = np.array([self.n_i_try(i, zs) for zs in z_prime])
            r_array = np.array([self.r(zs, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h) for zs in z_prime]) 
            integrand = n_array * (1 - (r_true / r_array)) * delta
            result.append(np.sum(integrand))
        return np.array(result)

In [76]:
class Cosmic_Shear:
    def __init__(self, cosmic_params):
        self.len = cosmic_params['len']
        self.z = cosmic_params['z']
        self.l = cosmic_params['l']
    
    def E2(self, z, w0, wa, Omega_m0, Omega_b0, Omega_DE0):
        Oomega_m0 = 2*(Omega_c0 + Omega_nu0 + Omega_b0) - (Omega_m0)
        Omega_k0 = 1 - (Oomega_m0+Omega_DE0)
        radicando = (Oomega_m0)*(1+z)**3 + Omega_DE0*(1+z)**(3*(1+wa+w0))*np.exp(-3*wa*(z / (1+z))) + (Omega_k0)*(1+z)**2
        return np.sqrt(radicando)
    
    def inverse_E2(self, z, w0, wa, Omega_m0, Omega_b0, Omega_DE0):
        return 1 / self.E2(z, w0, wa, Omega_m0, Omega_b0, Omega_DE0)
    
    def r(self, z, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h):
        H_0 = 100*h
        z_prime = np.linspace(0, z, self.len)
        delta = z / len(z_prime)
        integrand = self.inverse_E2(z_prime, w0, wa, Omega_m0, Omega_b0, Omega_DE0)*delta
        return np.sum(integrand)*(c/H_0)

    def SN(self, i, j):
        if i == j:
            return (0.3**2)/(35454308.58)
        else:
            return 0
        
    def D(self, z, Omega_m0, gamma):
        z_prime = np.linspace(0, z, 15)
        E_array = self.E2(z_prime, w0, wa, Omega_m0, Omega_b0, Omega_DE0)
        Omega_m = (Omega_m0*(1+z_prime)**3)/(E_array**2)
        delta = z / len(z_prime)
        integral = np.sum(((Omega_m**(gamma))/(1+z_prime))*delta)
        return np.exp(-integral)
    
    ## Define de Matter Power Spectrum

    interp_func = RectBivariateSpline(z_array, np.log10(k_array), np.log10(P_array_2d))

    def PK(self, z, k):
        return 10**(self.interp_func(z, np.log10(k), grid=False))

    def PPS(self, z, l, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h):
        k = ((10**l + 0.5)/self.r(z, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h))
        P = float(self.PK(z, k))
        return P
    
    def PSS_beyond(self, z, l, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        k = ((10**l + 0.5)/self.r(z, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h))
        D_0 = self.D(0, Omega_m0, gamma)
        P = float(self.PK(z, k))*((self.D(z, Omega_m0, gamma)/D_0)**2)
        return P

    # Otras derivadas
    
    interp_func_ns = RectBivariateSpline(z_array, np.log10(k_array), der_P_ns)

    def der_PK_ns(self, z, k):
        '''
        P with k
        '''
        return self.interp_func_ns(z, np.log10(k), grid=False)

    def der_PPS_ns(self, z, l, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        k = ((10**l + 0.5)/self.r(z, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h))
        P = float(self.der_PK_ns(z, k))
        return P

    # Interpolacion der with sigma_8

    interp_func_sig = RectBivariateSpline(z_array, np.log10(k_array), der_P_sigma8)

    def der_PK_sig(self, z, k):
        return self.interp_func_sig(z, np.log10(k), grid=False)

    def der_PPS_sig(self, z, l, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        k = ((10**l + 0.5)/self.r(z, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h))
        P = float(self.der_PK_sig(z, k))
        return P

    def Cosmic_Shear_l3(self, i ,j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        Aia = 1.72
        Cia = 0.0134
        H_0 = 100*h
        z_max = 2.5
        z_min = 0.001
        z_prime = self.z
        delta = (z_max - z_min)/len(z_prime)
        SNs = self.SN(i, j)

        params = {'len': 15 , 'z': z_prime}
        
        A = Class_integration(params)

        result = []

        D_array = np.array([self.D(zs, Omega_m0, gamma) for zs in z_prime])
        E_array = self.E2(z_prime, w0, wa, Omega_m0, Omega_b0, Omega_DE0)
        Wi = np.array(A.Window2(i, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h))
        n_i_array = np.array([A.n_i_try(i, zs) for zs in z_prime])
        Wj = np.array(A.Window2(j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h))
        n_j_array = np.array([A.n_i_try(j, zs) for zs in z_prime])
        r_array = np.array([self.r(zs, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h) for zs in z_prime])

        operador1 = ((((1.5*Omega_m0)**2) * (H_0 / c)**3)*(1+z_prime)**2)
        operador2 = 1.5*Omega_m0*(1+z_prime)*((H_0 / c)**3)
        operador3 =  (H_0 / c)**3

        K_gg = operador1*(Wi*Wj/(E_array))
        K_Ig =operador2*((n_i_array*Wj) + (n_j_array*Wi))/(r_array/c/H_0)
        K_II = operador3*(n_i_array*n_j_array*E_array)/((r_array/c/H_0)**2)
        
        for ls in self.l:

            P_gg = np.array([self.PPS(z_primes, ls, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h) for z_primes in z_prime])
            P_Ig = (-Aia*Cia*Omega_m0/D_array)*np.array([self.PPS(z_primes, ls, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h)  for z_primes in z_prime])
            P_II = ((-Aia*Cia*Omega_m0/D_array)**2)*np.array([self.PPS(z_primes, ls, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h)  for z_primes in z_prime])

            integrand = ((K_gg*P_gg) + (K_Ig*P_Ig) + (K_II*P_II))*float(delta)
            integral =  np.sum(integrand)

            integral = integral + SNs

            result.append(integral)

        return np.array(result)
    
    def Der_C_Omega(self, i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        C_plus = self.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0*(1+epsilon), Omega_b0, Omega_DE0, h, gamma)
        C_minus = self.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0*(1-epsilon), Omega_b0, Omega_DE0, h, gamma)

        return (C_plus - C_minus)/(epsilon*2*Omega_m0)

    def Der_C_h(self, i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        C_plus = self.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h*(1+epsilon), gamma)
        C_minus = self.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h*(1-epsilon), gamma)

        return (C_plus - C_minus)/(epsilon*2*h)

    def Der_C_Omega_b(self, i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        C_plus = self.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0, Omega_b0*(1+epsilon), Omega_DE0, h, gamma)
        C_minus = self.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0, Omega_b0*(1-epsilon), Omega_DE0, h, gamma)

        return (C_plus - C_minus)/(epsilon*2*Omega_b0)

    def Der_C_Omega_DE(self, i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        C_plus = self.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0, Omega_b0, Omega_DE0*(1+epsilon), h, gamma)
        C_minus = self.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0, Omega_b0, Omega_DE0*(1-epsilon), h, gamma)

        return (C_plus - C_minus)/(epsilon*2*Omega_DE0)

    def Der_C_w0(self, i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        C_plus = self.Cosmic_Shear_l3(i, j, w0*(1+epsilon), wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
        C_minus = self.Cosmic_Shear_l3(i, j, w0*(1-epsilon), wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)

        A = -(C_plus - C_minus)/(epsilon*2*w0)

        return A

    def Der_C_wa(self, i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        C_plus = self.Cosmic_Shear_l3(i, j, w0, wa + epsilon, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
        C_minus = self.Cosmic_Shear_l3(i, j, w0, wa - epsilon, Omega_m0, Omega_b0, Omega_DE0, h, gamma)

        return (C_plus - C_minus)/(epsilon*2)
    
    def Der_C_ns(self, i ,j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma): ## FALTAN TERMINOS
        '''
        Integrate the derivative of P
        '''
        Aia = 1.72
        Cia = 0.0134
        H_0 = 100*h
        z_max = 2.5
        z_min = 0.001
        z_prime = self.z
        delta = (z_max - z_min)/len(z_prime)
        SNs = self.SN(i, j)

        params = {'len': 15 , 'z': z_prime}
        
        A = Class_integration(params)

        result = []

        D_array = np.array([self.D(zs, Omega_m0, gamma) for zs in z_prime])
        E_array = self.E2(z_prime, w0, wa, Omega_m0, Omega_b0, Omega_DE0)
        Wi = np.array(A.Window2(i, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h))
        n_i_array = np.array([A.n_i_try(i, zs) for zs in z_prime])
        Wj = np.array(A.Window2(j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h))
        n_j_array = np.array([A.n_i_try(j, zs) for zs in z_prime])
        r_array = np.array([self.r(zs, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h) for zs in z_prime])

        operador1 = ((((1.5*Omega_m0)**2) * (H_0 / c)**3)*(1+z_prime)**2)
        operador2 = 1.5*Omega_m0*(1+z_prime)*((H_0 / c)**3)
        operador3 =  (H_0 / c)**3

        K_gg = operador1*(Wi*Wj/(E_array))
        K_Ig =operador2*((n_i_array*Wj) + (n_j_array*Wi))/(r_array/c/H_0)
        K_II = operador3*(n_i_array*n_j_array*E_array)/((r_array/c/H_0)**2)
        
        for ls in self.l:

            P_gg = np.array([self.der_PPS_ns(z_primes, ls, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma) for z_primes in z_prime])
            P_Ig = (-Aia*Cia*Omega_m0/D_array)*np.array([self.der_PPS_ns(z_primes, ls, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)  for z_primes in z_prime])
            P_II = ((-Aia*Cia*Omega_m0/D_array)**2)*np.array([self.der_PPS_ns(z_primes, ls, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma) for z_primes in z_prime])

            integrand = ((K_gg*P_gg) + (K_Ig*P_Ig) + (K_II*P_II))*float(delta)
            integral =  np.sum(integrand)

            integral = integral + SNs

            result.append(integral)

        return np.array(result)
    
    def Der_C_sig(self, i ,j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        '''
        Integrate the derivative of P
        '''
        Aia = 1.72
        Cia = 0.0134
        H_0 = 100*h
        z_max = 2.5
        z_min = 0.001
        z_prime = self.z
        delta = (z_max - z_min)/len(z_prime)
        SNs = self.SN(i, j)

        params = {'len': 15 , 'z': z_prime}
        
        A = Class_integration(params)

        result = []

        D_array = np.array([self.D(zs, Omega_m0, gamma) for zs in z_prime])
        E_array = self.E2(z_prime, w0, wa, Omega_m0, Omega_b0, Omega_DE0)
        Wi = np.array(A.Window2(i, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h))
        n_i_array = np.array([A.n_i_try(i, zs) for zs in z_prime])
        Wj = np.array(A.Window2(j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h))
        n_j_array = np.array([A.n_i_try(j, zs) for zs in z_prime])
        r_array = np.array([self.r(zs, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h) for zs in z_prime])

        operador1 = ((((1.5*Omega_m0)**2) * (H_0 / c)**3)*(1+z_prime)**2)
        operador2 = 1.5*Omega_m0*(1+z_prime)*((H_0 / c)**3)
        operador3 =  (H_0 / c)**3

        K_gg = operador1*(Wi*Wj/(E_array))
        K_Ig =operador2*((n_i_array*Wj) + (n_j_array*Wi))/(r_array/c/H_0)
        K_II = operador3*(n_i_array*n_j_array*E_array)/((r_array/c/H_0)**2)
        
        for ls in self.l:

            P_gg = np.array([self.der_PPS_sig(z_primes, ls, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma) for z_primes in z_prime])
            P_Ig = np.array([(-Aia*Cia*Omega_m0/D_array)*self.der_PPS_sig(z_primes, ls, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)  for z_primes in z_prime])
            P_II = np.array([((-Aia*Cia*Omega_m0/D_array)**2)*self.der_PPS_sig(z_primes, ls, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma) for z_primes in z_prime])

            integrand = ((K_gg*P_gg) + (K_Ig*P_Ig) + (K_II*P_II))*float(delta)
            integral =  np.sum(integrand)

            integral = integral + SNs

            result.append(integral)

        return np.array(result)
    
    def Der_C_gamma(self, i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        '''
        Integra las funciones derivadas
        '''
        
        C_plus = self.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma*(1+epsilon))
        C_minus = self.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma*(1-epsilon))

        return (C_plus - C_minus)/(epsilon*2*gamma)

    ##
        
    def Gran_Derivative(self, i, j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, epsilon, parametro1):
        if parametro1 == 'Om_m':
            return self.Der_C_Omega(i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
        elif parametro1 == 'Om_b':
            return self.Der_C_Omega_b(i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
        elif parametro1 == 'h':
            return self.Der_C_h(i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
        elif parametro1 == 'Om_DE':
            return self.Der_C_Omega_DE(i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
        elif parametro1 == 'w0':
            return self.Der_C_w0(i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
        elif parametro1 == 'wa':
            return self.Der_C_wa(i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
        elif parametro1 == 'ns':
            return self.Der_C_ns(i ,j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
        elif parametro1 == 'sigma8':
            return self.Der_C_sig(i ,j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
        elif parametro1 == 'gamma':
            return self.Der_C_gamma(i ,j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)

In [80]:
class Fisher:
    def __init__(self, params):
        self.num = params['num_par']

    def trace(self, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h):
        L_array = np.log10(np.logspace(np.log10(10), np.log10(1500), 100))
        cosmic_params = {'l': L_array, 'len': 15 , 'z': np.linspace(0.001, 2.5, 10)}
        A = Cosmic_Shear(cosmic_params)
        
        f_sky = 0.3636
        C = np.zeros((10, 10), dtype=object)
        dC_dq_Om_m_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_h_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_Om_b_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_ns_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_sig_matrix = np.zeros((10, 10), dtype=object)
        
        for i in range(10):
            for j in range(i, 10):
                logging.info(f"Calculated pair: {(i, j)}")
                C[i, j] = C[j, i] = A.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
                dC_dq_Om_m_matrix[i, j] = dC_dq_Om_m_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'Om_m')
                dC_dq_h_matrix[i, j] = dC_dq_h_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'h')
                dC_dq_Om_b_matrix[i, j] = dC_dq_Om_b_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'Om_b')
                dC_dq_ns_matrix[i, j] =  dC_dq_ns_matrix[j, i]  = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'ns')
                dC_dq_sig_matrix[i, j] = dC_dq_sig_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'sigma8')

        C_matrix = np.array(C.tolist(), dtype=float)
        C_matrix_2d = np.squeeze(C_matrix)

        dC_dq_Om_m_matrix = np.array(dC_dq_Om_m_matrix.tolist(), dtype=float)
        dC_dq_h_matrix = np.array(dC_dq_h_matrix.tolist(), dtype=float)
        dC_dq_Om_b_matrix = np.array(dC_dq_Om_b_matrix.tolist(), dtype=float)
        dC_dq_ns_matrix = np.array(dC_dq_ns_matrix.tolist(), dtype=float)
        dC_dq_sig_matrix = np.array(dC_dq_sig_matrix.tolist(), dtype=float)

        dC_dq_Om_m_matrix_2d = np.squeeze(dC_dq_Om_m_matrix)
        dC_dq_h_matrix_2d = np.squeeze(dC_dq_h_matrix)
        dC_dq_Om_b_matrix_2d = np.squeeze(dC_dq_Om_b_matrix)
        dC_dq_ns_matrix_2d = np.squeeze(dC_dq_ns_matrix)
        dC_dq_sig_matrix_2d = np.squeeze(dC_dq_sig_matrix)

        F = np.zeros((self.num, self.num))
        
        for i, l in enumerate(L_array):

            L_parameter = ((2*(10**(l)) + 1)/2)
            C_inv = np.linalg.inv(C_matrix_2d[:, :, i])


            # OMEGA_M 
            Om_m_Om_m = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv)
            Om_m_h = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_h_matrix_2d[:, :, i] @ C_inv)
            Om_m_Om_b = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv)
            Om_m_ns = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            Om_m_sig = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)

            # H
            h_h = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_h_matrix_2d[:, :, i] @ C_inv)
            h_Om_b = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv)
            h_ns = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            h_sig = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)

            # OMEGA_B
            Om_b_Om_b = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv)
            Om_b_ns = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            Om_b_sig = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)

            # NS
            ns_ns = np.trace(dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            ns_sig = np.trace(dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)

            # SIGMA8
            sig_sig = np.trace(dC_dq_sig_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)    

            F[0, 0] += (L_parameter)*Om_m_Om_m
            F[0, 1] += (L_parameter)*Om_m_h
            F[0, 2] += (L_parameter)*Om_m_Om_b
            F[0, 3] += (L_parameter)*Om_m_ns
            F[0, 4] += (L_parameter)*Om_m_sig

            F[1, 1] += (L_parameter)*h_h
            F[1, 2] += (L_parameter)*h_Om_b
            F[1, 3] += (L_parameter)*h_ns
            F[1, 4] += (L_parameter)*h_sig

            F[2, 2] += (L_parameter)*Om_b_Om_b
            F[2, 3] += (L_parameter)*Om_b_ns
            F[2, 4] += (L_parameter)*Om_b_sig

            F[3, 3] += (L_parameter)*ns_ns
            F[3, 4] += (L_parameter)*ns_sig

            F[4, 4] += (L_parameter)*sig_sig

        Fisher = F + F.T - np.diag(F.diagonal())
        logging.info(f"Fisher Matrix: {Fisher}")

        return f_sky*Fisher

In [81]:
params = {'num_par': 5}

A = Fisher(params)

In [82]:
# Time: 1.41 sec :)

F = A.trace(w0, wa, Omega_m0, Omega_b0, Omega_DE0, h)

INFO:root:Calculated pair: (0, 0)
INFO:root:Calculated pair: (0, 1)
INFO:root:Calculated pair: (0, 2)
INFO:root:Calculated pair: (0, 3)
INFO:root:Calculated pair: (0, 4)
INFO:root:Calculated pair: (0, 5)
INFO:root:Calculated pair: (0, 6)
INFO:root:Calculated pair: (0, 7)
INFO:root:Calculated pair: (0, 8)
INFO:root:Calculated pair: (0, 9)
INFO:root:Calculated pair: (1, 1)
INFO:root:Calculated pair: (1, 2)
INFO:root:Calculated pair: (1, 3)
INFO:root:Calculated pair: (1, 4)
INFO:root:Calculated pair: (1, 5)
INFO:root:Calculated pair: (1, 6)
INFO:root:Calculated pair: (1, 7)
INFO:root:Calculated pair: (1, 8)
INFO:root:Calculated pair: (1, 9)
INFO:root:Calculated pair: (2, 2)
INFO:root:Calculated pair: (2, 3)
INFO:root:Calculated pair: (2, 4)
INFO:root:Calculated pair: (2, 5)
INFO:root:Calculated pair: (2, 6)
INFO:root:Calculated pair: (2, 7)
INFO:root:Calculated pair: (2, 8)
INFO:root:Calculated pair: (2, 9)
INFO:root:Calculated pair: (3, 3)
INFO:root:Calculated pair: (3, 4)
INFO:root:Calc

In [85]:
Cov = np.linalg.inv(F)

In [86]:
print('Error Omega_m: ' + str(np.sqrt(Cov[0, 0])))
print('Error h: ' + str(np.sqrt(Cov[1, 1])))
print('Error Omega_b: ' + str(np.sqrt(Cov[2, 2])))
print('Error ns: ' + str(np.sqrt(Cov[3, 3])))
print('Error sigma8: ' + str(np.sqrt(Cov[4, 4])))

Error Omega_m: 0.0037303744670278962
Error h: 0.003450028450705611
Error Omega_b: 0.006709149121692588
Error ns: 0.00474148088959634
Error sigma8: 0.0006835657870658082


In [88]:
# Relative errors

err_Om_m = np.sqrt(Cov[0, 0]) / Omega_m0
err_h = np.sqrt(Cov[1, 1]) / h
err_Om_b = np.sqrt(Cov[2, 2]) / Omega_b0
err_ns = np.sqrt(Cov[3, 3]) / ns
err_sig = np.sqrt(Cov[4, 4]) / sigma8

print('Error Relativo Omega_m: ' + str(err_Om_m))
print('Error Relativo h: ' + str(err_h))
print('Error Relativo Omega_b: ' + str(err_Om_b))
print('Error Relativo ns: ' + str(err_ns))
print('Error Relativo sigma8: ' + str(err_sig))

Error Relativo Omega_m: 0.011657420069761068
Error Relativo h: 0.005149296195083001
Error Relativo Omega_b: 0.13418298243385174
Error Relativo ns: 0.004939042593329521
Error Relativo sigma8: 0.0008377031704237845


In [90]:
## Comparison with the paper (the numbers must be under 10)

S_Om_m = 0.018
S_h = 0.21
S_Om_b = 0.47
S_ns = 0.035
S_sig = 0.0087

def comparison(created, expected):
    return 100*(1 - (created/expected))

print('Comparison for Omega_m: ' + str(comparison(err_Om_m, S_Om_m)))
print('Comparison for h: ' + str(comparison(err_h, S_h)))
print('Comparison for Omega_b: ' + str(comparison(err_Om_b, S_Om_b)))
print('Comparison for ns: ' + str(comparison(err_ns, S_ns)))
print('Comparison for sigma8: ' + str(comparison(err_sig, S_sig)))

Comparison for Omega_m: 35.236555167994055
Comparison for h: 97.54795419281763
Comparison for Omega_b: 71.45042926939324
Comparison for ns: 85.88844973334423
Comparison for sigma8: 90.3712279261634


# $\Lambda CDM$ non-flat

Parameters:

1. $\Omega_{m,0}$

2. $h$

3. $\Omega_{b,0}$

4. $\Omega{DE,0}$

5. $n_s$

6. $\sigma_9$

In [92]:
class Fisher_2:
    def __init__(self, params):
        self.num = params['num_par']

    def trace(self, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h):
        L_array = np.log10(np.logspace(np.log10(10), np.log10(1500), 100))
        cosmic_params = {'l': L_array, 'len': 15 , 'z': np.linspace(0.001, 2.5, 10)}
        A = Cosmic_Shear(cosmic_params)
        
        f_sky = 0.3636
        C = np.zeros((10, 10), dtype=object)
        dC_dq_Om_m_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_h_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_Om_b_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_Om_DE_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_ns_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_sig_matrix = np.zeros((10, 10), dtype=object)
        
        for i in range(10):
            for j in range(i, 10):
                logging.info(f"Calculated pair: {(i, j)}")
                C[i, j] = C[j, i] = A.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
                dC_dq_Om_m_matrix[i, j] = dC_dq_Om_m_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'Om_m')
                dC_dq_h_matrix[i, j] = dC_dq_h_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'h')
                dC_dq_Om_b_matrix[i, j] = dC_dq_Om_b_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'Om_b')
                dC_dq_Om_DE_matrix[i, j] = dC_dq_Om_DE_matrix[j, i] =  A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'Om_DE')
                dC_dq_ns_matrix[i, j] =  dC_dq_ns_matrix[j, i]  = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'ns')
                dC_dq_sig_matrix[i, j] = dC_dq_sig_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'sigma8')

        C_matrix = np.array(C.tolist(), dtype=float)
        C_matrix_2d = np.squeeze(C_matrix)

        dC_dq_Om_m_matrix = np.array(dC_dq_Om_m_matrix.tolist(), dtype=float)
        dC_dq_h_matrix = np.array(dC_dq_h_matrix.tolist(), dtype=float)
        dC_dq_Om_b_matrix = np.array(dC_dq_Om_b_matrix.tolist(), dtype=float)
        dC_dq_Om_DE_matrix = np.array(dC_dq_Om_DE_matrix.tolist(), dtype=float)
        dC_dq_ns_matrix = np.array(dC_dq_ns_matrix.tolist(), dtype=float)
        dC_dq_sig_matrix = np.array(dC_dq_sig_matrix.tolist(), dtype=float)

        dC_dq_Om_m_matrix_2d = np.squeeze(dC_dq_Om_m_matrix)
        dC_dq_h_matrix_2d = np.squeeze(dC_dq_h_matrix)
        dC_dq_Om_b_matrix_2d = np.squeeze(dC_dq_Om_b_matrix)
        dC_dq_Om_DE_matrix_2d = np.squeeze(dC_dq_Om_DE_matrix)
        dC_dq_ns_matrix_2d = np.squeeze(dC_dq_ns_matrix)
        dC_dq_sig_matrix_2d = np.squeeze(dC_dq_sig_matrix)

        F = np.zeros((self.num, self.num))
        
        for i, l in enumerate(L_array):

            L_parameter = ((2*(10**(l)) + 1)/2)
            C_inv = np.linalg.inv(C_matrix_2d[:, :, i])


            # OMEGA_M 
            Om_m_Om_m = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv)
            Om_m_h = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_h_matrix_2d[:, :, i] @ C_inv)
            Om_m_Om_b = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv)
            Om_m_Om_DE = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv)
            Om_m_ns = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            Om_m_sig = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)

            # H
            h_h = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_h_matrix_2d[:, :, i] @ C_inv)
            h_Om_b = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv)
            h_Om_DE = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv)
            h_ns = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            h_sig = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)

            # OMEGA_B
            Om_b_Om_b = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv)
            Om_b_Om_DE = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv)
            Om_b_ns = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            Om_b_sig = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)

            # OMEGA_DE
            Om_DE_Om_DE = np.trace(dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv)
            Om_DE_ns = np.trace(dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            Om_DE_sig = np.trace(dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)

            # NS
            ns_ns = np.trace(dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            ns_sig = np.trace(dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)

            # SIGMA8
            sig_sig = np.trace(dC_dq_sig_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)    

            F[0, 0] += (L_parameter)*Om_m_Om_m
            F[0, 1] += (L_parameter)*Om_m_h
            F[0, 2] += (L_parameter)*Om_m_Om_b
            F[0, 3] += (L_parameter)*Om_m_Om_DE
            F[0, 4] += (L_parameter)*Om_m_ns
            F[0, 5] += (L_parameter)*Om_m_sig

            F[1, 1] += (L_parameter)*h_h
            F[1, 2] += (L_parameter)*h_Om_b
            F[1, 3] += (L_parameter)*h_Om_DE
            F[1, 4] += (L_parameter)*h_ns
            F[1, 5] += (L_parameter)*h_sig

            F[2, 2] += (L_parameter)*Om_b_Om_b
            F[2, 3] += (L_parameter)*Om_b_Om_DE
            F[2, 4] += (L_parameter)*Om_b_ns
            F[2, 5] += (L_parameter)*Om_b_sig

            F[3, 3] += (L_parameter)*Om_DE_Om_DE
            F[3, 4] += (L_parameter)*Om_DE_ns
            F[3, 5] += (L_parameter)*Om_DE_sig

            F[4, 4] += (L_parameter)*ns_ns
            F[4, 5] += (L_parameter)*ns_sig

            F[5, 5] += (L_parameter)*sig_sig

        Fisher = F + F.T - np.diag(F.diagonal())
        logging.info(f"Fisher Matrix: {Fisher}")

        return f_sky*Fisher

In [95]:
params = {'num_par': 6}

A = Fisher_2(params)

In [96]:
# Time: 1.58 sec

F = A.trace(w0, wa, Omega_m0, Omega_b0, Omega_DE0, h)

INFO:root:Calculated pair: (0, 0)
INFO:root:Calculated pair: (0, 1)
INFO:root:Calculated pair: (0, 2)
INFO:root:Calculated pair: (0, 3)
INFO:root:Calculated pair: (0, 4)
INFO:root:Calculated pair: (0, 5)
INFO:root:Calculated pair: (0, 6)
INFO:root:Calculated pair: (0, 7)
INFO:root:Calculated pair: (0, 8)
INFO:root:Calculated pair: (0, 9)
INFO:root:Calculated pair: (1, 1)
INFO:root:Calculated pair: (1, 2)
INFO:root:Calculated pair: (1, 3)
INFO:root:Calculated pair: (1, 4)
INFO:root:Calculated pair: (1, 5)
INFO:root:Calculated pair: (1, 6)
INFO:root:Calculated pair: (1, 7)
INFO:root:Calculated pair: (1, 8)
INFO:root:Calculated pair: (1, 9)
INFO:root:Calculated pair: (2, 2)
INFO:root:Calculated pair: (2, 3)
INFO:root:Calculated pair: (2, 4)
INFO:root:Calculated pair: (2, 5)
INFO:root:Calculated pair: (2, 6)
INFO:root:Calculated pair: (2, 7)
INFO:root:Calculated pair: (2, 8)
INFO:root:Calculated pair: (2, 9)
INFO:root:Calculated pair: (3, 3)
INFO:root:Calculated pair: (3, 4)
INFO:root:Calc

In [101]:
Cov = np.linalg.inv(F)

In [102]:
print('Error Omega_m: ' + str(np.sqrt(Cov[0, 0])))
print('Error h: ' + str(np.sqrt(Cov[1, 1])))
print('Error Omega_b: ' + str(np.sqrt(Cov[2, 2])))
print('Error Omega_DE: ' + str(np.sqrt(Cov[3, 3])))
print('Error ns: ' + str(np.sqrt(Cov[4, 4])))
print('Error sigma8: ' + str(np.sqrt(Cov[5, 5])))

Error Omega_m: 0.006301936118444214
Error h: 0.009141183797300499
Error Omega_b: 0.018994711641613937
Error Omega_DE: 0.08436297373005758
Error ns: 0.004789540685755464
Error sigma8: 0.000835357824744487


In [104]:
# Relative errors

err_Om_m = np.sqrt(Cov[0, 0]) / Omega_m0
err_h = np.sqrt(Cov[1, 1]) / h
err_Om_b = np.sqrt(Cov[2, 2]) / Omega_b0
err_Om_DE = np.sqrt(Cov[3, 3]) / Omega_DE0
err_ns = np.sqrt(Cov[4, 4]) / ns
err_sig = np.sqrt(Cov[5, 5]) / sigma8

print('Error Relativo Omega_m: ' + str(err_Om_m))
print('Error Relativo h: ' + str(err_h))
print('Error Relativo Omega_b: ' + str(err_Om_b))
print('Error Relativo Omega_DE: ' + str(err_Om_DE))
print('Error Relativo ns: ' + str(err_ns))
print('Error Relativo sigma8: ' + str(err_sig))

Error Relativo Omega_m: 0.019693550134133053
Error Relativo h: 0.013643557906418654
Error Relativo Omega_b: 0.3798942328322787
Error Relativo Omega_DE: 0.12406319666184937
Error Relativo ns: 0.004989104880995275
Error Relativo sigma8: 0.0010237228244417734


In [107]:
## Comparison with the paper (the numbers must be under 10)

S_Om_m = 0.021
S_h = 0.21
S_Om_b = 0.47
S_Om_DE = 0.035
S_ns = 0.035
S_sig = 0.015


print('Comparison for Omega_m: ' + str(comparison(err_Om_m, S_Om_m)))
print('Comparison for h: ' + str(comparison(err_h, S_h)))
print('Comparison for Omega_b: ' + str(comparison(err_Om_b, S_Om_b)))
print('Comparison for Omega_DE: ' + str(comparison(err_Om_DE, S_Om_DE)))
print('Comparison for ns: ' + str(comparison(err_ns, S_ns)))
print('Comparison for sigma8: ' + str(comparison(err_sig, S_sig)))

Comparison for Omega_m: 6.22118983746166
Comparison for h: 93.50306766361017
Comparison for Omega_b: 19.171439822919425
Comparison for Omega_DE: 254.46627617671243
Comparison for ns: 85.74541462572779
Comparison for sigma8: 93.17518117038817


# $w_0$ and $w_a$ flat

Parameters:

1. $\Omega_{m,0}$

2. $h$

3. $\Omega_{b,0}$

4. $w_0$

5. $w_a$

6. $n_s$

7. $\sigma_8$

In [109]:
class Fisher_3:
    def __init__(self, params):
        self.num = params['num_par']

    def trace(self, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h):
        L_array = np.log10(np.logspace(np.log10(10), np.log10(1500), 100))
        cosmic_params = {'l': L_array, 'len': 15 , 'z': np.linspace(0.001, 2.5, 10)}
        A = Cosmic_Shear(cosmic_params)
        
        f_sky = 0.3636
        C = np.zeros((10, 10), dtype=object)
        dC_dq_Om_m_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_h_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_Om_b_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_w0_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_wa_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_ns_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_sig_matrix = np.zeros((10, 10), dtype=object)
        
        for i in range(10):
            for j in range(i, 10):
                logging.info(f"Calculated pair: {(i, j)}")
                C[i, j] = C[j, i] = A.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
                dC_dq_Om_m_matrix[i, j] = dC_dq_Om_m_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'Om_m')
                dC_dq_h_matrix[i, j] = dC_dq_h_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'h')
                dC_dq_Om_b_matrix[i, j] = dC_dq_Om_b_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'Om_b')
                dC_dq_w0_matrix[i, j] = dC_dq_w0_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'w0')
                dC_dq_wa_matrix[i, j] = dC_dq_wa_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'wa')
                dC_dq_ns_matrix[i, j] =  dC_dq_ns_matrix[j, i]  = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'ns')
                dC_dq_sig_matrix[i, j] = dC_dq_sig_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'sigma8')

        C_matrix = np.array(C.tolist(), dtype=float)
        C_matrix_2d = np.squeeze(C_matrix)

        dC_dq_Om_m_matrix = np.array(dC_dq_Om_m_matrix.tolist(), dtype=float)
        dC_dq_h_matrix = np.array(dC_dq_h_matrix.tolist(), dtype=float)
        dC_dq_Om_b_matrix = np.array(dC_dq_Om_b_matrix.tolist(), dtype=float)
        dC_dq_w0_matrix = np.array(dC_dq_w0_matrix.tolist(), dtype=float)
        dC_dq_wa_matrix = np.array(dC_dq_wa_matrix.tolist(), dtype=float)
        dC_dq_ns_matrix = np.array(dC_dq_ns_matrix.tolist(), dtype=float)
        dC_dq_sig_matrix = np.array(dC_dq_sig_matrix.tolist(), dtype=float)

        dC_dq_Om_m_matrix_2d = np.squeeze(dC_dq_Om_m_matrix)
        dC_dq_h_matrix_2d = np.squeeze(dC_dq_h_matrix)
        dC_dq_Om_b_matrix_2d = np.squeeze(dC_dq_Om_b_matrix)
        dC_dq_w0_matrix_2d = np.squeeze(dC_dq_w0_matrix)
        dC_dq_wa_matrix_2d = np.squeeze(dC_dq_wa_matrix)
        dC_dq_ns_matrix_2d = np.squeeze(dC_dq_ns_matrix)
        dC_dq_sig_matrix_2d = np.squeeze(dC_dq_sig_matrix)

        F = np.zeros((self.num, self.num))
        
        for i, l in enumerate(L_array):

            L_parameter = ((2*(10**(l)) + 1)/2)
            C_inv = np.linalg.inv(C_matrix_2d[:, :, i])


            # OMEGA_M 
            Om_m_Om_m = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv)
            Om_m_h = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_h_matrix_2d[:, :, i] @ C_inv)
            Om_m_Om_b = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv)
            Om_m_w0 = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_w0_matrix_2d[:, :, i] @ C_inv)
            Om_m_wa = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            Om_m_ns = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            Om_m_sig = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)

            # H
            h_h = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_h_matrix_2d[:, :, i] @ C_inv)
            h_Om_b = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv)
            h_w0 = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_w0_matrix_2d[:, :, i] @ C_inv)
            h_wa = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            h_ns = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            h_sig = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)

            # OMEGA_B
            Om_b_Om_b = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv)
            Om_b_w0 = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_w0_matrix_2d[:, :, i] @ C_inv)
            Om_b_wa = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            Om_b_ns = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            Om_b_sig = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)

            # W0
            w0_w0 = np.trace(dC_dq_w0_matrix_2d[:, :, i] @ C_inv @ dC_dq_w0_matrix_2d[:, :, i] @ C_inv)
            w0_wa = np.trace(dC_dq_w0_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            w0_ns = np.trace(dC_dq_w0_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            w0_sig = np.trace(dC_dq_w0_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)

            # WA
            wa_wa = np.trace(dC_dq_wa_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            wa_ns = np.trace(dC_dq_wa_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            wa_sig = np.trace(dC_dq_wa_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)           

            # NS
            ns_ns = np.trace(dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            ns_sig = np.trace(dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)

            # SIGMA8
            sig_sig = np.trace(dC_dq_sig_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)    

            F[0, 0] += (L_parameter)*Om_m_Om_m
            F[0, 1] += (L_parameter)*Om_m_h
            F[0, 2] += (L_parameter)*Om_m_Om_b
            F[0, 3] += (L_parameter)*Om_m_w0
            F[0, 4] += (L_parameter)*Om_m_wa
            F[0, 5] += (L_parameter)*Om_m_ns
            F[0, 6] += (L_parameter)*Om_m_sig

            F[1, 1] += (L_parameter)*h_h
            F[1, 2] += (L_parameter)*h_Om_b
            F[1, 3] += (L_parameter)*h_w0
            F[1, 4] += (L_parameter)*h_wa
            F[1, 5] += (L_parameter)*h_ns
            F[1, 6] += (L_parameter)*h_sig

            F[2, 2] += (L_parameter)*Om_b_Om_b
            F[2, 3] += (L_parameter)*Om_b_w0
            F[2, 4] += (L_parameter)*Om_b_wa
            F[2, 5] += (L_parameter)*Om_b_ns
            F[2, 6] += (L_parameter)*Om_b_sig

            F[3, 3] += (L_parameter)*w0_w0
            F[3, 4] += (L_parameter)*w0_wa
            F[3, 5] += (L_parameter)*w0_ns
            F[3, 6] += (L_parameter)*w0_sig

            F[4, 4] += (L_parameter)*wa_wa
            F[4, 5] += (L_parameter)*wa_ns
            F[4, 6] += (L_parameter)*wa_sig      

            F[5, 5] += (L_parameter)*ns_ns
            F[5, 6] += (L_parameter)*ns_sig

            F[6, 6] += (L_parameter)*sig_sig

        Fisher = F + F.T - np.diag(F.diagonal())
        logging.info(f"Fisher Matrix: {Fisher}")

        return f_sky*Fisher

In [110]:
params = {'num_par': 7}

A = Fisher_3(params)

In [111]:
# Time: 2.21 sec

F = A.trace(w0, wa, Omega_m0, Omega_b0, Omega_DE0, h)

INFO:root:Calculated pair: (0, 0)
INFO:root:Calculated pair: (0, 1)
INFO:root:Calculated pair: (0, 2)
INFO:root:Calculated pair: (0, 3)
INFO:root:Calculated pair: (0, 4)
INFO:root:Calculated pair: (0, 5)
INFO:root:Calculated pair: (0, 6)
INFO:root:Calculated pair: (0, 7)
INFO:root:Calculated pair: (0, 8)
INFO:root:Calculated pair: (0, 9)
INFO:root:Calculated pair: (1, 1)
INFO:root:Calculated pair: (1, 2)
INFO:root:Calculated pair: (1, 3)
INFO:root:Calculated pair: (1, 4)
INFO:root:Calculated pair: (1, 5)
INFO:root:Calculated pair: (1, 6)
INFO:root:Calculated pair: (1, 7)
INFO:root:Calculated pair: (1, 8)
INFO:root:Calculated pair: (1, 9)
INFO:root:Calculated pair: (2, 2)
INFO:root:Calculated pair: (2, 3)
INFO:root:Calculated pair: (2, 4)
INFO:root:Calculated pair: (2, 5)
INFO:root:Calculated pair: (2, 6)
INFO:root:Calculated pair: (2, 7)
INFO:root:Calculated pair: (2, 8)
INFO:root:Calculated pair: (2, 9)
INFO:root:Calculated pair: (3, 3)
INFO:root:Calculated pair: (3, 4)
INFO:root:Calc

In [118]:
Cov = np.linalg.inv(F)

print('ERRORS')

print('Error Omega_m: ' + str(np.sqrt(Cov[0, 0])))
print('Error h: ' + str(np.sqrt(Cov[1, 1])))
print('Error Omega_b: ' + str(np.sqrt(Cov[2, 2])))
print('Error w0: ' + str(np.sqrt(Cov[3, 3])))
print('Error wa: ' + str(np.sqrt(Cov[4, 4])))
print('Error ns: ' + str(np.sqrt(Cov[5, 5])))
print('Error sigma8: ' + str(np.sqrt(Cov[6, 6])))

print(' ')
print('RELATIVE ERRORS')

err_Om_m = np.sqrt(Cov[0, 0]) / Omega_m0
err_h = np.sqrt(Cov[1, 1]) / h
err_Om_b = np.sqrt(Cov[2, 2]) / Omega_b0
err_w0 = np.sqrt(Cov[3, 3])
err_wa = np.sqrt(Cov[4, 4])
err_ns = np.sqrt(Cov[5, 5]) / ns
err_sig = np.sqrt(Cov[6, 6]) / sigma8

print('Error Relativo Omega_m: ' + str(err_Om_m))
print('Error Relativo h: ' + str(err_h))
print('Error Relativo Omega_b: ' + str(err_Om_b))
print('Error Relativo w0: ' + str(err_w0))
print('Error Relativo wa: ' + str(err_wa))
print('Error Relativo ns: ' + str(err_ns))
print('Error Relativo sigma8: ' + str(err_sig))

print(' ')
print('COMPARISON')

## Comparison with the paper (the numbers must be under 10)

S_Om_m = 0.044
S_h = 0.21
S_Om_b = 0.47
S_w0 = 0.16
S_wa = 0.59
S_ns = 0.038
S_sig = 0.019

print('Comparison for Omega_m: ' + str(comparison(err_Om_m, S_Om_m)))
print('Comparison for h: ' + str(comparison(err_h, S_h)))
print('Comparison for Omega_b: ' + str(comparison(err_Om_b, S_Om_b)))
print('Comparison for w0 (considering w0 = 1): ' + str(comparison(err_w0, 1)))
print('Comparison for wa (dont considering the function): ' + str(err_wa))
print('Comparison for ns: ' + str(comparison(err_ns, S_ns)))
print('Comparison for sigma8: ' + str(comparison(err_sig, S_sig)))

ERRORS
Error Omega_m: 0.01373734966299395
Error h: 0.019460757828549554
Error Omega_b: 0.016794628409912325
Error w0: 0.31049498114743146
Error wa: 0.9330125678345871
Error ns: 0.0047923971534798325
Error sigma8: 0.0008448466074366017
 
RELATIVE ERRORS
Error Relativo Omega_m: 0.04292921718239757
Error Relativo h: 0.029045907206790376
Error Relativo Omega_b: 0.3358925681982465
Error Relativo w0: 0.31049498114743146
Error Relativo wa: 0.9330125678345871
Error Relativo ns: 0.004992080368208159
Error Relativo sigma8: 0.0010353512346036786
 
COMPARISON
Comparison for Omega_m: 2.4335973127327937
Comparison for h: 86.1686156158141
Comparison for Omega_b: 28.533496128032667
Comparison for w0 (considering w0 = 1): 68.95050188525686
Comparison for wa (dont considering the function): 0.9330125678345871
Comparison for ns: 86.86294639945221
Comparison for sigma8: 94.5507829757701


# $w_0$ and $w_a$ non-flat

Parameters:

1. $\Omega_{m,0}$

2. $h$

3. $\Omega_{b,0}$

4. $\Omega_{DE,0}$

5. $w_0$

6. $w_a$

7. $n_s$

8. $\sigma_8$

In [121]:
class Fisher_4:
    def __init__(self, params):
        self.num = params['num_par']

    def trace(self, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h):
        L_array = np.log10(np.logspace(np.log10(10), np.log10(1500), 100))
        cosmic_params = {'l': L_array, 'len': 15 , 'z': np.linspace(0.001, 2.5, 10)}
        A = Cosmic_Shear(cosmic_params)
        
        f_sky = 0.3636
        C = np.zeros((10, 10), dtype=object)
        dC_dq_Om_m_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_h_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_Om_b_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_Om_DE_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_w0_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_wa_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_ns_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_sig_matrix = np.zeros((10, 10), dtype=object)
        
        for i in range(10):
            for j in range(i, 10):
                logging.info(f"Calculated pair: {(i, j)}")
                C[i, j] = C[j, i] = A.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
                dC_dq_Om_m_matrix[i, j] = dC_dq_Om_m_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'Om_m')
                dC_dq_h_matrix[i, j] = dC_dq_h_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'h')
                dC_dq_Om_b_matrix[i, j] = dC_dq_Om_b_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'Om_b')
                dC_dq_Om_DE_matrix[i, j] = dC_dq_Om_DE_matrix[j, i] =  A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'Om_DE')
                dC_dq_w0_matrix[i, j] = dC_dq_w0_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'w0')
                dC_dq_wa_matrix[i, j] = dC_dq_wa_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'wa')
                dC_dq_ns_matrix[i, j] =  dC_dq_ns_matrix[j, i]  = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'ns')
                dC_dq_sig_matrix[i, j] = dC_dq_sig_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'sigma8')

        C_matrix = np.array(C.tolist(), dtype=float)
        C_matrix_2d = np.squeeze(C_matrix)

        dC_dq_Om_m_matrix = np.array(dC_dq_Om_m_matrix.tolist(), dtype=float)
        dC_dq_h_matrix = np.array(dC_dq_h_matrix.tolist(), dtype=float)
        dC_dq_Om_b_matrix = np.array(dC_dq_Om_b_matrix.tolist(), dtype=float)
        dC_dq_Om_DE_matrix = np.array(dC_dq_Om_DE_matrix.tolist(), dtype=float)
        dC_dq_w0_matrix = np.array(dC_dq_w0_matrix.tolist(), dtype=float)
        dC_dq_wa_matrix = np.array(dC_dq_wa_matrix.tolist(), dtype=float)
        dC_dq_ns_matrix = np.array(dC_dq_ns_matrix.tolist(), dtype=float)
        dC_dq_sig_matrix = np.array(dC_dq_sig_matrix.tolist(), dtype=float)

        dC_dq_Om_m_matrix_2d = np.squeeze(dC_dq_Om_m_matrix)
        dC_dq_h_matrix_2d = np.squeeze(dC_dq_h_matrix)
        dC_dq_Om_b_matrix_2d = np.squeeze(dC_dq_Om_b_matrix)
        dC_dq_Om_DE_matrix_2d = np.squeeze(dC_dq_Om_DE_matrix)
        dC_dq_w0_matrix_2d = np.squeeze(dC_dq_w0_matrix)
        dC_dq_wa_matrix_2d = np.squeeze(dC_dq_wa_matrix)
        dC_dq_ns_matrix_2d = np.squeeze(dC_dq_ns_matrix)
        dC_dq_sig_matrix_2d = np.squeeze(dC_dq_sig_matrix)

        F = np.zeros((self.num, self.num))
        
        for i, l in enumerate(L_array):

            L_parameter = ((2*(10**(l)) + 1)/2)
            C_inv = np.linalg.inv(C_matrix_2d[:, :, i])


            # OMEGA_M 
            Om_m_Om_m = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv)
            Om_m_h = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_h_matrix_2d[:, :, i] @ C_inv)
            Om_m_Om_b = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv)
            Om_m_Om_DE = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv)
            Om_m_w0 = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_w0_matrix_2d[:, :, i] @ C_inv)
            Om_m_wa = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            Om_m_ns = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            Om_m_sig = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)

            # H
            h_h = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_h_matrix_2d[:, :, i] @ C_inv)
            h_Om_b = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv)
            h_Om_DE = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv)
            h_w0 = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_w0_matrix_2d[:, :, i] @ C_inv)
            h_wa = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            h_ns = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            h_sig = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)

            # OMEGA_B
            Om_b_Om_b = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv)
            Om_b_Om_DE = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv)
            Om_b_w0 = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_w0_matrix_2d[:, :, i] @ C_inv)
            Om_b_wa = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            Om_b_ns = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            Om_b_sig = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)

            # OMEGA_DE
            Om_DE_Om_DE = np.trace(dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv)
            Om_DE_w0 = np.trace(dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv @ dC_dq_w0_matrix_2d[:, :, i] @ C_inv)
            Om_DE_wa = np.trace(dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            Om_DE_ns = np.trace(dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            Om_DE_sig = np.trace(dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)

            # W0
            w0_w0 = np.trace(dC_dq_w0_matrix_2d[:, :, i] @ C_inv @ dC_dq_w0_matrix_2d[:, :, i] @ C_inv)
            w0_wa = np.trace(dC_dq_w0_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            w0_ns = np.trace(dC_dq_w0_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            w0_sig = np.trace(dC_dq_w0_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)

            # WA
            wa_wa = np.trace(dC_dq_wa_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            wa_ns = np.trace(dC_dq_wa_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            wa_sig = np.trace(dC_dq_wa_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)           

            # NS
            ns_ns = np.trace(dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            ns_sig = np.trace(dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)

            # SIGMA8
            sig_sig = np.trace(dC_dq_sig_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)    

            F[0, 0] += (L_parameter)*Om_m_Om_m
            F[0, 1] += (L_parameter)*Om_m_h
            F[0, 2] += (L_parameter)*Om_m_Om_b
            F[0, 3] += (L_parameter)*Om_m_Om_DE
            F[0, 4] += (L_parameter)*Om_m_w0
            F[0, 5] += (L_parameter)*Om_m_wa
            F[0, 6] += (L_parameter)*Om_m_ns
            F[0, 7] += (L_parameter)*Om_m_sig

            F[1, 1] += (L_parameter)*h_h
            F[1, 2] += (L_parameter)*h_Om_b
            F[1, 3] += (L_parameter)*h_Om_DE
            F[1, 4] += (L_parameter)*h_w0
            F[1, 5] += (L_parameter)*h_wa
            F[1, 6] += (L_parameter)*h_ns
            F[1, 7] += (L_parameter)*h_sig

            F[2, 2] += (L_parameter)*Om_b_Om_b
            F[2, 3] += (L_parameter)*Om_b_Om_DE
            F[2, 4] += (L_parameter)*Om_b_w0
            F[2, 5] += (L_parameter)*Om_b_wa
            F[2, 6] += (L_parameter)*Om_b_ns
            F[2, 7] += (L_parameter)*Om_b_sig

            F[3, 3] += (L_parameter)*Om_DE_Om_DE
            F[3, 4] += (L_parameter)*Om_DE_w0
            F[3, 5] += (L_parameter)*Om_DE_wa
            F[3, 6] += (L_parameter)*Om_DE_ns
            F[3, 7] += (L_parameter)*Om_DE_sig

            F[4, 4] += (L_parameter)*w0_w0
            F[4, 5] += (L_parameter)*w0_wa
            F[4, 6] += (L_parameter)*w0_ns
            F[4, 7] += (L_parameter)*w0_sig

            F[5, 5] += (L_parameter)*wa_wa
            F[5, 6] += (L_parameter)*wa_ns
            F[5, 7] += (L_parameter)*wa_sig      

            F[6, 6] += (L_parameter)*ns_ns
            F[6, 7] += (L_parameter)*ns_sig

            F[7, 7] += (L_parameter)*sig_sig

        Fisher = F + F.T - np.diag(F.diagonal())
        logging.info(f"Fisher Matrix: {Fisher}")

        return f_sky*Fisher

In [122]:
params = {'num_par': 8}

A = Fisher_4(params)

In [123]:
# Time: 2.41 sec

F = A.trace(w0, wa, Omega_m0, Omega_b0, Omega_DE0, h)

INFO:root:Calculated pair: (0, 0)
INFO:root:Calculated pair: (0, 1)
INFO:root:Calculated pair: (0, 2)
INFO:root:Calculated pair: (0, 3)
INFO:root:Calculated pair: (0, 4)
INFO:root:Calculated pair: (0, 5)
INFO:root:Calculated pair: (0, 6)
INFO:root:Calculated pair: (0, 7)
INFO:root:Calculated pair: (0, 8)
INFO:root:Calculated pair: (0, 9)
INFO:root:Calculated pair: (1, 1)
INFO:root:Calculated pair: (1, 2)
INFO:root:Calculated pair: (1, 3)
INFO:root:Calculated pair: (1, 4)
INFO:root:Calculated pair: (1, 5)
INFO:root:Calculated pair: (1, 6)
INFO:root:Calculated pair: (1, 7)
INFO:root:Calculated pair: (1, 8)
INFO:root:Calculated pair: (1, 9)
INFO:root:Calculated pair: (2, 2)
INFO:root:Calculated pair: (2, 3)
INFO:root:Calculated pair: (2, 4)
INFO:root:Calculated pair: (2, 5)
INFO:root:Calculated pair: (2, 6)
INFO:root:Calculated pair: (2, 7)
INFO:root:Calculated pair: (2, 8)
INFO:root:Calculated pair: (2, 9)
INFO:root:Calculated pair: (3, 3)
INFO:root:Calculated pair: (3, 4)
INFO:root:Calc

In [125]:
Cov = np.linalg.inv(F)

print('ERRORS')

print('Error Omega_m: ' + str(np.sqrt(Cov[0, 0])))
print('Error h: ' + str(np.sqrt(Cov[1, 1])))
print('Error Omega_b: ' + str(np.sqrt(Cov[2, 2])))
print('Error Omega_DE: ' + str(np.sqrt(Cov[3, 3])))
print('Error w0: ' + str(np.sqrt(Cov[4, 4])))
print('Error wa: ' + str(np.sqrt(Cov[5, 5])))
print('Error ns: ' + str(np.sqrt(Cov[6, 6])))
print('Error sigma8: ' + str(np.sqrt(Cov[7, 7])))

print(' ')
print('RELATIVE ERRORS')

err_Om_m = np.sqrt(Cov[0, 0]) / Omega_m0
err_h = np.sqrt(Cov[1, 1]) / h
err_Om_b = np.sqrt(Cov[2, 2]) / Omega_b0
err_Om_DE = np.sqrt(Cov[3, 3]) / Omega_DE0
err_w0 = np.sqrt(Cov[4, 4])
err_wa = np.sqrt(Cov[5, 5])
err_ns = np.sqrt(Cov[6, 6]) / ns
err_sig = np.sqrt(Cov[7, 7]) / sigma8

print('Error Relativo Omega_m: ' + str(err_Om_m))
print('Error Relativo h: ' + str(err_h))
print('Error Relativo Omega_b: ' + str(err_Om_b))
print('Error Relativo Omega_DE: ' + str(err_Om_DE))
print('Error Relativo w0: ' + str(err_w0))
print('Error Relativo wa: ' + str(err_wa))
print('Error Relativo ns: ' + str(err_ns))
print('Error Relativo sigma8: ' + str(err_sig))

print(' ')
print('COMPARISON')

## Comparison with the paper (the numbers must be under 10)

S_Om_m = 0.047
S_h = 0.21
S_Om_b = 0.48
S_Om_DE = 0.2
S_w0 = 0.23
S_wa = 1.85
S_ns = 0.038
S_sig = 0.02

print('Comparison for Omega_m: ' + str(comparison(err_Om_m, S_Om_m)))
print('Comparison for h: ' + str(comparison(err_h, S_h)))
print('Comparison for Omega_b: ' + str(comparison(err_Om_b, S_Om_b)))
print('Comparison for w0 (considering w0 = 1): ' + str(comparison(err_w0, 1)))
print('Comparison for wa (dont considering the function): ' + str(err_wa))
print('Comparison for ns: ' + str(comparison(err_ns, S_ns)))
print('Comparison for sigma8: ' + str(comparison(err_sig, S_sig)))

ERRORS
Error Omega_m: 0.023297173482518432
Error h: 0.032900817074823445
Error Omega_b: 0.2176442106038275
Error Omega_DE: 2.010590672087681
Error w0: 1.155324780141744
Error wa: 6.24135716194841
Error ns: 0.004821550137387941
Error sigma8: 0.0008813877319608617
 
RELATIVE ERRORS
Error Relativo Omega_m: 0.0728036662603998
Error Relativo h: 0.049105697126602156
Error Relativo Omega_b: 4.35288421207655
Error Relativo Omega_DE: 2.9567509883642367
Error Relativo w0: 1.155324780141744
Error Relativo wa: 6.24135716194841
Error Relativo ns: 0.005022448059779106
Error Relativo sigma8: 0.0010801320244618405
 
COMPARISON
Comparison for Omega_m: 54.901417575318725
Comparison for h: 76.61633470161802
Comparison for Omega_b: 806.8508775159479
Comparison for w0 (considering w0 = 1): 15.532478014174389
Comparison for wa (dont considering the function): 6.24135716194841
Comparison for ns: 86.78303142163392
Comparison for sigma8: 94.5993398776908


# $w_0$, $w_a$ and $\gamma$ non-flat

Parameters:

1. $\Omega_{m,0}$

2. $h$

3. $\Omega_{b,0}$

4. $\Omega_{DE,0}$

5. $w_0$

6. $w_a$

7. $n_s$

8. $\sigma_8$

9. $\gamma$

In [132]:
class Cosmic_Shear:
    def __init__(self, cosmic_params):
        self.len = cosmic_params['len']
        self.z = cosmic_params['z']
        self.l = cosmic_params['l']
    
    def E2(self, z, w0, wa, Omega_m0, Omega_b0, Omega_DE0):
        Oomega_m0 = 2*(Omega_c0 + Omega_nu0 + Omega_b0) - (Omega_m0)
        Omega_k0 = 1 - (Oomega_m0+Omega_DE0)
        radicando = (Oomega_m0)*(1+z)**3 + Omega_DE0*(1+z)**(3*(1+wa+w0))*np.exp(-3*wa*(z / (1+z))) + (Omega_k0)*(1+z)**2
        return np.sqrt(radicando)
    
    def inverse_E2(self, z, w0, wa, Omega_m0, Omega_b0, Omega_DE0):
        return 1 / self.E2(z, w0, wa, Omega_m0, Omega_b0, Omega_DE0)
    
    def r(self, z, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h):
        H_0 = 100*h
        z_prime = np.linspace(0, z, self.len)
        delta = z / len(z_prime)
        integrand = self.inverse_E2(z_prime, w0, wa, Omega_m0, Omega_b0, Omega_DE0)*delta
        return np.sum(integrand)*(c/H_0)

    def SN(self, i, j):
        if i == j:
            return (0.3**2)/(35454308.58)
        else:
            return 0
        
    def D(self, z, Omega_m0, gamma):
        z_prime = np.linspace(0, z, 15)
        E_array = self.E2(z_prime, w0, wa, Omega_m0, Omega_b0, Omega_DE0)
        Omega_m = (Omega_m0*(1+z_prime)**3)/(E_array**2)
        delta = z / len(z_prime)
        integral = np.sum(((Omega_m**(gamma))/(1+z_prime))*delta)
        return np.exp(-integral)
    
    ## Define de Matter Power Spectrum

    interp_func = RectBivariateSpline(z_array, np.log10(k_array), np.log10(P_array_2d))

    def PK(self, z, k):
        return 10**(self.interp_func(z, np.log10(k), grid=False))

    def PPS(self, z, l, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h):
        k = ((10**l + 0.5)/self.r(z, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h))
        P = float(self.PK(z, k))
        return P
    
    def PPS_beyond(self, z, l, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        k = ((10**l + 0.5)/self.r(z, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h))
        D_0 = self.D(0, Omega_m0, gamma)
        P = float(self.PK(z, k))*((self.D(z, Omega_m0, gamma)/D_0)**2)
        return P

    # Otras derivadas
    
    interp_func_ns = RectBivariateSpline(z_array, np.log10(k_array), der_P_ns)

    def der_PK_ns(self, z, k):
        '''
        P with k
        '''
        return self.interp_func_ns(z, np.log10(k), grid=False)

    def der_PPS_ns_beyond(self, z, l, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        k = ((10**l + 0.5)/self.r(z, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h))
        D_0 = self.D(0, Omega_m0, gamma)
        P = float(self.der_PK_ns(z, k))*((self.D(z, Omega_m0, gamma)/D_0)**2)
        return P

    # Interpolacion der with sigma_8

    interp_func_sig = RectBivariateSpline(z_array, np.log10(k_array), der_P_sigma8)

    def der_PK_sig(self, z, k):
        return self.interp_func_sig(z, np.log10(k), grid=False)

    def der_PPS_sig_beyond(self, z, l, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        k = ((10**l + 0.5)/self.r(z, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h))
        D_0 = self.D(0, Omega_m0, gamma)
        P = float(self.der_PK_sig(z, k))*((self.D(z, Omega_m0, gamma)/D_0)**2)
        return P

    def Cosmic_Shear_l3(self, i ,j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        Aia = 1.72
        Cia = 0.0134
        H_0 = 100*h
        z_max = 2.5
        z_min = 0.001
        z_prime = self.z
        delta = (z_max - z_min)/len(z_prime)
        SNs = self.SN(i, j)

        params = {'len': 15 , 'z': z_prime}
        
        A = Class_integration(params)

        result = []

        D_array = np.array([self.D(zs, Omega_m0, gamma) for zs in z_prime])
        E_array = self.E2(z_prime, w0, wa, Omega_m0, Omega_b0, Omega_DE0)
        Wi = np.array(A.Window2(i, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h))
        n_i_array = np.array([A.n_i_try(i, zs) for zs in z_prime])
        Wj = np.array(A.Window2(j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h))
        n_j_array = np.array([A.n_i_try(j, zs) for zs in z_prime])
        r_array = np.array([self.r(zs, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h) for zs in z_prime])

        operador1 = ((((1.5*Omega_m0)**2) * (H_0 / c)**3)*(1+z_prime)**2)
        operador2 = 1.5*Omega_m0*(1+z_prime)*((H_0 / c)**3)
        operador3 =  (H_0 / c)**3

        K_gg = operador1*(Wi*Wj/(E_array))
        K_Ig =operador2*((n_i_array*Wj) + (n_j_array*Wi))/(r_array/c/H_0)
        K_II = operador3*(n_i_array*n_j_array*E_array)/((r_array/c/H_0)**2)
        
        for ls in self.l:

            P_gg = np.array([self.PPS_beyond(z_primes, ls, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma) for z_primes in z_prime])
            P_Ig = (-Aia*Cia*Omega_m0/D_array)*np.array([self.PPS_beyond(z_primes, ls, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)  for z_primes in z_prime])
            P_II = ((-Aia*Cia*Omega_m0/D_array)**2)*np.array([self.PPS_beyond(z_primes, ls, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)  for z_primes in z_prime])

            integrand = ((K_gg*P_gg) + (K_Ig*P_Ig) + (K_II*P_II))*float(delta)
            integral =  np.sum(integrand)

            integral = integral + SNs

            result.append(integral)

        return np.array(result)
    
    def Der_C_Omega(self, i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        C_plus = self.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0*(1+epsilon), Omega_b0, Omega_DE0, h, gamma)
        C_minus = self.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0*(1-epsilon), Omega_b0, Omega_DE0, h, gamma)

        return (C_plus - C_minus)/(epsilon*2*Omega_m0)

    def Der_C_h(self, i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        C_plus = self.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h*(1+epsilon), gamma)
        C_minus = self.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h*(1-epsilon), gamma)

        return (C_plus - C_minus)/(epsilon*2*h)

    def Der_C_Omega_b(self, i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        C_plus = self.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0, Omega_b0*(1+epsilon), Omega_DE0, h, gamma)
        C_minus = self.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0, Omega_b0*(1-epsilon), Omega_DE0, h, gamma)

        return (C_plus - C_minus)/(epsilon*2*Omega_b0)

    def Der_C_Omega_DE(self, i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        C_plus = self.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0, Omega_b0, Omega_DE0*(1+epsilon), h, gamma)
        C_minus = self.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0, Omega_b0, Omega_DE0*(1-epsilon), h, gamma)

        return (C_plus - C_minus)/(epsilon*2*Omega_DE0)

    def Der_C_w0(self, i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        C_plus = self.Cosmic_Shear_l3(i, j, w0*(1+epsilon), wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
        C_minus = self.Cosmic_Shear_l3(i, j, w0*(1-epsilon), wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)

        A = -(C_plus - C_minus)/(epsilon*2*w0)

        return A

    def Der_C_wa(self, i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        C_plus = self.Cosmic_Shear_l3(i, j, w0, wa + epsilon, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
        C_minus = self.Cosmic_Shear_l3(i, j, w0, wa - epsilon, Omega_m0, Omega_b0, Omega_DE0, h, gamma)

        return (C_plus - C_minus)/(epsilon*2)
    
    def Der_C_ns(self, i ,j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma): 
        '''
        Integrate the derivative of P
        '''
        Aia = 1.72
        Cia = 0.0134
        H_0 = 100*h
        z_max = 2.5
        z_min = 0.001
        z_prime = self.z
        delta = (z_max - z_min)/len(z_prime)
        SNs = self.SN(i, j)

        params = {'len': 15 , 'z': z_prime}
        
        A = Class_integration(params)

        result = []

        D_array = np.array([self.D(zs, Omega_m0, gamma) for zs in z_prime])
        E_array = self.E2(z_prime, w0, wa, Omega_m0, Omega_b0, Omega_DE0)
        Wi = np.array(A.Window2(i, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h))
        n_i_array = np.array([A.n_i_try(i, zs) for zs in z_prime])
        Wj = np.array(A.Window2(j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h))
        n_j_array = np.array([A.n_i_try(j, zs) for zs in z_prime])
        r_array = np.array([self.r(zs, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h) for zs in z_prime])

        operador1 = ((((1.5*Omega_m0)**2) * (H_0 / c)**3)*(1+z_prime)**2)
        operador2 = 1.5*Omega_m0*(1+z_prime)*((H_0 / c)**3)
        operador3 =  (H_0 / c)**3

        K_gg = operador1*(Wi*Wj/(E_array))
        K_Ig =operador2*((n_i_array*Wj) + (n_j_array*Wi))/(r_array/c/H_0)
        K_II = operador3*(n_i_array*n_j_array*E_array)/((r_array/c/H_0)**2)
        
        for ls in self.l:

            P_gg = np.array([self.der_PPS_ns_beyond(z_primes, ls, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma) for z_primes in z_prime])
            P_Ig = (-Aia*Cia*Omega_m0/D_array)*np.array([self.der_PPS_ns_beyond(z_primes, ls, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)  for z_primes in z_prime])
            P_II = ((-Aia*Cia*Omega_m0/D_array)**2)*np.array([self.der_PPS_ns_beyond(z_primes, ls, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma) for z_primes in z_prime])

            integrand = ((K_gg*P_gg) + (K_Ig*P_Ig) + (K_II*P_II))*float(delta)
            integral =  np.sum(integrand)

            integral = integral + SNs

            result.append(integral)

        return np.array(result)
    
    def Der_C_sig(self, i ,j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        '''
        Integrate the derivative of P
        '''
        Aia = 1.72
        Cia = 0.0134
        H_0 = 100*h
        z_max = 2.5
        z_min = 0.001
        z_prime = self.z
        delta = (z_max - z_min)/len(z_prime)
        SNs = self.SN(i, j)

        params = {'len': 15 , 'z': z_prime}
        
        A = Class_integration(params)

        result = []

        D_array = np.array([self.D(zs, Omega_m0, gamma) for zs in z_prime])
        E_array = self.E2(z_prime, w0, wa, Omega_m0, Omega_b0, Omega_DE0)
        Wi = np.array(A.Window2(i, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h))
        n_i_array = np.array([A.n_i_try(i, zs) for zs in z_prime])
        Wj = np.array(A.Window2(j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h))
        n_j_array = np.array([A.n_i_try(j, zs) for zs in z_prime])
        r_array = np.array([self.r(zs, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h) for zs in z_prime])

        operador1 = ((((1.5*Omega_m0)**2) * (H_0 / c)**3)*(1+z_prime)**2)
        operador2 = 1.5*Omega_m0*(1+z_prime)*((H_0 / c)**3)
        operador3 =  (H_0 / c)**3

        K_gg = operador1*(Wi*Wj/(E_array))
        K_Ig =operador2*((n_i_array*Wj) + (n_j_array*Wi))/(r_array/c/H_0)
        K_II = operador3*(n_i_array*n_j_array*E_array)/((r_array/c/H_0)**2)
        
        for ls in self.l:

            P_gg = np.array([self.der_PPS_sig_beyond(z_primes, ls, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma) for z_primes in z_prime])
            P_Ig = np.array([(-Aia*Cia*Omega_m0/D_array)*self.der_PPS_sig_beyond(z_primes, ls, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)  for z_primes in z_prime])
            P_II = np.array([((-Aia*Cia*Omega_m0/D_array)**2)*self.der_PPS_sig_beyond(z_primes, ls, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma) for z_primes in z_prime])

            integrand = ((K_gg*P_gg) + (K_Ig*P_Ig) + (K_II*P_II))*float(delta)
            integral =  np.sum(integrand)

            integral = integral + SNs

            result.append(integral)

        return np.array(result)
    
    def Der_C_gamma(self, i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        '''
        Integra las funciones derivadas
        '''
        
        C_plus = self.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma*(1+epsilon))
        C_minus = self.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma*(1-epsilon))

        return (C_plus - C_minus)/(epsilon*2*gamma)

    ##
        
    def Gran_Derivative(self, i, j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, epsilon, parametro1):
        if parametro1 == 'Om_m':
            return self.Der_C_Omega(i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
        elif parametro1 == 'Om_b':
            return self.Der_C_Omega_b(i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
        elif parametro1 == 'h':
            return self.Der_C_h(i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
        elif parametro1 == 'Om_DE':
            return self.Der_C_Omega_DE(i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
        elif parametro1 == 'w0':
            return self.Der_C_w0(i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
        elif parametro1 == 'wa':
            return self.Der_C_wa(i, j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
        elif parametro1 == 'ns':
            return self.Der_C_ns(i ,j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
        elif parametro1 == 'sigma8':
            return self.Der_C_sig(i ,j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
        elif parametro1 == 'gamma':
            return self.Der_C_gamma(i ,j, epsilon, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)

In [133]:
class Fisher_5:
    def __init__(self, params):
        self.num = params['num_par']

    def trace(self, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma):
        L_array = np.log10(np.logspace(np.log10(10), np.log10(1500), 100))
        cosmic_params = {'l': L_array, 'len': 15 , 'z': np.linspace(0.001, 2.5, 10)}
        A = Cosmic_Shear(cosmic_params)
        
        f_sky = 0.3636
        C = np.zeros((10, 10), dtype=object)
        dC_dq_Om_m_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_h_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_Om_b_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_Om_DE_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_w0_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_wa_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_ns_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_sig_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_gamma_matrix = np.zeros((10, 10), dtype=object)
        
        for i in range(10):
            for j in range(i, 10):
                logging.info(f"Calculated pair: {(i, j)}")
                C[i, j] = C[j, i] = A.Cosmic_Shear_l3(i, j, w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)
                dC_dq_Om_m_matrix[i, j] = dC_dq_Om_m_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'Om_m')
                dC_dq_h_matrix[i, j] = dC_dq_h_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'h')
                dC_dq_Om_b_matrix[i, j] = dC_dq_Om_b_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'Om_b')
                dC_dq_Om_DE_matrix[i, j] = dC_dq_Om_DE_matrix[j, i] =  A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'Om_DE')
                dC_dq_w0_matrix[i, j] = dC_dq_w0_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'w0')
                dC_dq_wa_matrix[i, j] = dC_dq_wa_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'wa')
                dC_dq_ns_matrix[i, j] =  dC_dq_ns_matrix[j, i]  = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'ns')
                dC_dq_sig_matrix[i, j] = dC_dq_sig_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'sigma8')
                dC_dq_gamma_matrix[i, j] = dC_dq_gamma_matrix[j, i] = A.Gran_Derivative(i, j,  w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma, 0.01, 'gamma')

        C_matrix = np.array(C.tolist(), dtype=float)
        C_matrix_2d = np.squeeze(C_matrix)

        dC_dq_Om_m_matrix = np.array(dC_dq_Om_m_matrix.tolist(), dtype=float)
        dC_dq_h_matrix = np.array(dC_dq_h_matrix.tolist(), dtype=float)
        dC_dq_Om_b_matrix = np.array(dC_dq_Om_b_matrix.tolist(), dtype=float)
        dC_dq_Om_DE_matrix = np.array(dC_dq_Om_DE_matrix.tolist(), dtype=float)
        dC_dq_w0_matrix = np.array(dC_dq_w0_matrix.tolist(), dtype=float)
        dC_dq_wa_matrix = np.array(dC_dq_wa_matrix.tolist(), dtype=float)
        dC_dq_ns_matrix = np.array(dC_dq_ns_matrix.tolist(), dtype=float)
        dC_dq_sig_matrix = np.array(dC_dq_sig_matrix.tolist(), dtype=float)
        dC_dq_gamma_matrix = np.array(dC_dq_gamma_matrix.tolist(), dtype=float)

        dC_dq_Om_m_matrix_2d = np.squeeze(dC_dq_Om_m_matrix)
        dC_dq_h_matrix_2d = np.squeeze(dC_dq_h_matrix)
        dC_dq_Om_b_matrix_2d = np.squeeze(dC_dq_Om_b_matrix)
        dC_dq_Om_DE_matrix_2d = np.squeeze(dC_dq_Om_DE_matrix)
        dC_dq_w0_matrix_2d = np.squeeze(dC_dq_w0_matrix)
        dC_dq_wa_matrix_2d = np.squeeze(dC_dq_wa_matrix)
        dC_dq_ns_matrix_2d = np.squeeze(dC_dq_ns_matrix)
        dC_dq_sig_matrix_2d = np.squeeze(dC_dq_sig_matrix)
        dC_dq_gamma_matrix_2d = np.squeeze(dC_dq_gamma_matrix)

        F = np.zeros((self.num, self.num))
        
        for i, l in enumerate(L_array):

            L_parameter = ((2*(10**(l)) + 1)/2)
            C_inv = np.linalg.inv(C_matrix_2d[:, :, i])

            # OMEGA_M 
            Om_m_Om_m = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv)
            Om_m_h = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_h_matrix_2d[:, :, i] @ C_inv)
            Om_m_Om_b = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv)
            Om_m_Om_DE = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv)
            Om_m_w0 = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_w0_matrix_2d[:, :, i] @ C_inv)
            Om_m_wa = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            Om_m_ns = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            Om_m_sig = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)
            Om_m_gamma = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_gamma_matrix_2d[:, :, i] @ C_inv)


            # H
            h_h = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_h_matrix_2d[:, :, i] @ C_inv)
            h_Om_b = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv)
            h_Om_DE = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv)
            h_w0 = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_w0_matrix_2d[:, :, i] @ C_inv)
            h_wa = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            h_ns = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            h_sig = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)
            h_gamma = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_gamma_matrix_2d[:, :, i] @ C_inv)

            # OMEGA_B
            Om_b_Om_b = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv)
            Om_b_Om_DE = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv)
            Om_b_w0 = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_w0_matrix_2d[:, :, i] @ C_inv)
            Om_b_wa = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            Om_b_ns = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            Om_b_sig = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)
            Om_b_gamma = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_gamma_matrix_2d[:, :, i] @ C_inv)

            # OMEGA_DE
            Om_DE_Om_DE = np.trace(dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv)
            Om_DE_w0 = np.trace(dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv @ dC_dq_w0_matrix_2d[:, :, i] @ C_inv)
            Om_DE_wa = np.trace(dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            Om_DE_ns = np.trace(dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            Om_DE_sig = np.trace(dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)
            Om_DE_gamma = np.trace(dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv @ dC_dq_gamma_matrix_2d[:, :, i] @ C_inv)

            # W0
            w0_w0 = np.trace(dC_dq_w0_matrix_2d[:, :, i] @ C_inv @ dC_dq_w0_matrix_2d[:, :, i] @ C_inv)
            w0_wa = np.trace(dC_dq_w0_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            w0_ns = np.trace(dC_dq_w0_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            w0_sig = np.trace(dC_dq_w0_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)
            w0_gamma = np.trace(dC_dq_w0_matrix_2d[:, :, i] @ C_inv @ dC_dq_gamma_matrix_2d[:, :, i] @ C_inv)

            # WA
            wa_wa = np.trace(dC_dq_wa_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            wa_ns = np.trace(dC_dq_wa_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            wa_sig = np.trace(dC_dq_wa_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv) 
            wa_gamma = np.trace(dC_dq_wa_matrix_2d[:, :, i] @ C_inv @ dC_dq_gamma_matrix_2d[:, :, i] @ C_inv)           

            # NS
            ns_ns = np.trace(dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            ns_sig = np.trace(dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)
            ns_gamma = np.trace(dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_gamma_matrix_2d[:, :, i] @ C_inv)

            # SIGMA8
            sig_sig = np.trace(dC_dq_sig_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv) 
            sig_gamma = np.trace(dC_dq_sig_matrix_2d[:, :, i] @ C_inv @ dC_dq_gamma_matrix_2d[:, :, i] @ C_inv)   

            # GAMMA
            gamma_gamma = np.trace(dC_dq_gamma_matrix_2d[:, :, i] @ C_inv @ dC_dq_gamma_matrix_2d[:, :, i] @ C_inv)  

            F[0, 0] += (L_parameter)*Om_m_Om_m
            F[0, 1] += (L_parameter)*Om_m_h
            F[0, 2] += (L_parameter)*Om_m_Om_b
            F[0, 3] += (L_parameter)*Om_m_Om_DE
            F[0, 4] += (L_parameter)*Om_m_w0
            F[0, 5] += (L_parameter)*Om_m_wa
            F[0, 6] += (L_parameter)*Om_m_ns
            F[0, 7] += (L_parameter)*Om_m_sig
            F[0, 8] += (L_parameter)*Om_m_gamma

            F[1, 1] += (L_parameter)*h_h
            F[1, 2] += (L_parameter)*h_Om_b
            F[1, 3] += (L_parameter)*h_Om_DE
            F[1, 4] += (L_parameter)*h_w0
            F[1, 5] += (L_parameter)*h_wa
            F[1, 6] += (L_parameter)*h_ns
            F[1, 7] += (L_parameter)*h_sig
            F[1, 8] += (L_parameter)*h_gamma

            F[2, 2] += (L_parameter)*Om_b_Om_b
            F[2, 3] += (L_parameter)*Om_b_Om_DE
            F[2, 4] += (L_parameter)*Om_b_w0
            F[2, 5] += (L_parameter)*Om_b_wa
            F[2, 6] += (L_parameter)*Om_b_ns
            F[2, 7] += (L_parameter)*Om_b_sig
            F[2, 8] += (L_parameter)*Om_b_gamma

            F[3, 3] += (L_parameter)*Om_DE_Om_DE
            F[3, 4] += (L_parameter)*Om_DE_w0
            F[3, 5] += (L_parameter)*Om_DE_wa
            F[3, 6] += (L_parameter)*Om_DE_ns
            F[3, 7] += (L_parameter)*Om_DE_sig
            F[3, 8] += (L_parameter)*Om_DE_gamma

            F[4, 4] += (L_parameter)*w0_w0
            F[4, 5] += (L_parameter)*w0_wa
            F[4, 6] += (L_parameter)*w0_ns
            F[4, 7] += (L_parameter)*w0_sig
            F[4, 8] += (L_parameter)*w0_gamma

            F[5, 5] += (L_parameter)*wa_wa
            F[5, 6] += (L_parameter)*wa_ns
            F[5, 7] += (L_parameter)*wa_sig 
            F[5, 8] += (L_parameter)*wa_gamma   

            F[6, 6] += (L_parameter)*ns_ns
            F[6, 7] += (L_parameter)*ns_sig
            F[6, 8] += (L_parameter)*ns_gamma

            F[7, 7] += (L_parameter)*sig_sig
            F[7, 8] += (L_parameter)*sig_gamma

            F[8, 8] += (L_parameter)*gamma_gamma

        Fisher = F + F.T - np.diag(F.diagonal())
        logging.info(f"Fisher Matrix: {Fisher}")

        return f_sky*Fisher

In [134]:
params = {'num_par': 9}

A = Fisher_5(params)

In [135]:
# Time: 8.47 sec...

F = A.trace(w0, wa, Omega_m0, Omega_b0, Omega_DE0, h, gamma)

INFO:root:Calculated pair: (0, 0)
INFO:root:Calculated pair: (0, 1)
INFO:root:Calculated pair: (0, 2)
INFO:root:Calculated pair: (0, 3)
INFO:root:Calculated pair: (0, 4)
INFO:root:Calculated pair: (0, 5)
INFO:root:Calculated pair: (0, 6)
INFO:root:Calculated pair: (0, 7)
INFO:root:Calculated pair: (0, 8)
INFO:root:Calculated pair: (0, 9)
INFO:root:Calculated pair: (1, 1)
INFO:root:Calculated pair: (1, 2)
INFO:root:Calculated pair: (1, 3)
INFO:root:Calculated pair: (1, 4)
INFO:root:Calculated pair: (1, 5)
INFO:root:Calculated pair: (1, 6)
INFO:root:Calculated pair: (1, 7)
INFO:root:Calculated pair: (1, 8)
INFO:root:Calculated pair: (1, 9)
INFO:root:Calculated pair: (2, 2)
INFO:root:Calculated pair: (2, 3)
INFO:root:Calculated pair: (2, 4)
INFO:root:Calculated pair: (2, 5)
INFO:root:Calculated pair: (2, 6)
INFO:root:Calculated pair: (2, 7)
INFO:root:Calculated pair: (2, 8)
INFO:root:Calculated pair: (2, 9)
INFO:root:Calculated pair: (3, 3)
INFO:root:Calculated pair: (3, 4)
INFO:root:Calc

In [136]:
Cov = np.linalg.inv(F)

print('ERRORS')

print('Error Omega_m: ' + str(np.sqrt(Cov[0, 0])))
print('Error h: ' + str(np.sqrt(Cov[1, 1])))
print('Error Omega_b: ' + str(np.sqrt(Cov[2, 2])))
print('Error Omega_DE: ' + str(np.sqrt(Cov[3, 3])))
print('Error w0: ' + str(np.sqrt(Cov[4, 4])))
print('Error wa: ' + str(np.sqrt(Cov[5, 5])))
print('Error ns: ' + str(np.sqrt(Cov[6, 6])))
print('Error sigma8: ' + str(np.sqrt(Cov[7, 7])))
print('Error gamma: ' + str(np.sqrt(Cov[8, 8])))

print(' ')
print('RELATIVE ERRORS')

err_Om_m = np.sqrt(Cov[0, 0]) / Omega_m0
err_h = np.sqrt(Cov[1, 1]) / h
err_Om_b = np.sqrt(Cov[2, 2]) / Omega_b0
err_Om_DE = np.sqrt(Cov[3, 3]) / Omega_DE0
err_w0 = np.sqrt(Cov[4, 4])
err_wa = np.sqrt(Cov[5, 5])
err_ns = np.sqrt(Cov[6, 6]) / ns
err_sig = np.sqrt(Cov[7, 7]) / sigma8
err_gamma = np.sqrt(Cov[8, 8]) / gamma

print('Error Relativo Omega_m: ' + str(err_Om_m))
print('Error Relativo h: ' + str(err_h))
print('Error Relativo Omega_b: ' + str(err_Om_b))
print('Error Relativo Omega_DE: ' + str(err_Om_DE))
print('Error Relativo w0: ' + str(err_w0))
print('Error Relativo wa: ' + str(err_wa))
print('Error Relativo ns: ' + str(err_ns))
print('Error Relativo sigma8: ' + str(err_sig))
print('Error Relativo gamma: ' + str(err_gamma))

print(' ')
print('COMPARISON')

## Comparison with the paper (the numbers must be under 10)

S_Om_m = 0.055
S_h = 0.23
S_Om_b = 0.5
S_Om_DE = 0.57
S_w0 = 0.25
S_wa = 1.9
S_ns = 0.037
S_sig = 0.029
S_gamma = 1.1

print('Comparison for Omega_m: ' + str(comparison(err_Om_m, S_Om_m)))
print('Comparison for h: ' + str(comparison(err_h, S_h)))
print('Comparison for Omega_b: ' + str(comparison(err_Om_b, S_Om_b)))
print('Comparison for w0 (considering w0 = 1): ' + str(comparison(err_w0, 1)))
print('Comparison for wa (dont considering the function): ' + str(err_wa))
print('Comparison for ns: ' + str(comparison(err_ns, S_ns)))
print('Comparison for sigma8: ' + str(comparison(err_sig, S_sig)))
print('Comparison for sigma8: ' + str(comparison(err_gamma, S_gamma)))

ERRORS
Error Omega_m: 0.07642391035494979
Error h: 0.08226752878777546
Error Omega_b: 0.24032251762794085
Error Omega_DE: 3.105997701085739
Error w0: 1.4879151434282594
Error wa: 11.15455279232989
Error ns: 0.004667886326410129
Error sigma8: 0.0004437842470567325
Error gamma: 0.5158823971990457
 
RELATIVE ERRORS
Error Relativo Omega_m: 0.23882471699717167
Error Relativo h: 0.12278735639966486
Error Relativo Omega_b: 4.806450352558817
Error Relativo Omega_DE: 4.567643678067263
Error Relativo w0: 1.4879151434282594
Error Relativo wa: 11.15455279232989
Error Relativo ns: 0.0048623815900105515
Error Relativo sigma8: 0.0005438532439420742
Error Relativo gamma: 0.9457843948649172
 
COMPARISON
Comparison for Omega_m: 334.2267581766758
Comparison for h: 46.614192869710934
Comparison for Omega_b: 861.2900705117634
Comparison for w0 (considering w0 = 1): 48.791514342825934
Comparison for wa (dont considering the function): 11.15455279232989
Comparison for ns: 86.8584281351066
Comparison for sigm